In [20]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
from scipy.optimize import minimize 

In [3]:
single_cell = pd.read_csv('Selected_gene_sc.tsv', sep = '\t')
single_cell

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.323555,0.094705,0.192635,0.577982,0.353719,0.128162,0.032882,0.104255,0.600437,0.131016,0.029491,0.132231,0.017921,0.362637,0.036290,0.015385,0.180328,0.076923,0.304348
1,0.000863,0.003055,0.000000,0.000000,0.001653,0.005059,0.001934,0.000000,0.000000,0.005348,0.000000,0.002755,0.853047,0.000000,0.274194,0.558974,0.032787,0.329670,0.000000
2,0.005177,0.006110,0.001416,0.009174,0.009917,0.008432,0.174081,0.002128,0.008734,0.002674,0.439678,0.002755,0.465950,0.014652,0.411290,0.358974,0.038251,0.153846,0.000000
3,0.153581,0.068228,0.264873,0.397554,0.171901,0.178752,0.166344,0.287234,0.257642,0.697861,0.083110,0.228650,0.111111,0.183150,0.032258,0.071795,0.180328,0.087912,0.130435
4,0.006903,1.009165,0.000000,0.000000,0.004959,0.005059,0.000000,0.000000,0.002183,0.002674,0.000000,0.000000,0.007168,0.000000,0.000000,0.000000,0.038251,0.000000,0.065217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,0.993097,0.945010,0.603399,0.749235,0.662810,0.598651,0.843327,0.525532,0.772926,0.548128,1.428954,0.586777,0.544803,0.501832,0.584677,0.707692,0.628415,0.802198,0.782609
2860,0.260569,0.423625,0.120397,0.152905,0.180165,0.139966,0.396518,0.140426,0.213974,0.125668,0.605898,0.195592,0.394265,0.139194,0.334677,0.205128,0.469945,0.164835,0.347826
2861,0.414150,0.120163,0.140227,0.501529,0.198347,0.124789,0.243714,0.306383,0.879913,0.141711,0.233244,0.088154,0.215054,0.461538,0.318548,0.128205,0.229508,0.197802,0.369565
2862,0.053494,0.177189,0.021246,0.079511,0.071074,0.057336,0.334623,0.072340,0.019651,0.136364,0.378016,0.066116,0.136201,0.051282,0.282258,0.076923,0.852459,0.109890,0.043478


In [10]:
bulk = pd.read_csv('Selected_gene_bulk.tsv', sep = '\t')
gene = bulk['id']
bulk_values = bulk.iloc[:,1:]
bulk_values

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110
0,0.272364,0.428677,0.108904,0.282876,0.443393,0.057185,0.277305,0.142332,0.119310,0.257542,...,0.297698,0.227899,0.058972,0.263744,0.202460,0.269000,0.340271,0.127299,0.322611,0.120151
1,0.651938,1.162660,0.593676,0.925709,1.217457,2.514403,1.598007,0.746156,0.639809,0.927875,...,1.868096,0.907949,2.120641,1.265974,2.091835,0.367555,0.927659,1.486896,0.618150,0.629630
2,0.014840,0.020105,0.031594,0.007659,0.001436,0.009095,0.003830,0.029679,0.008617,0.003351,...,0.012446,0.011010,0.359023,0.006223,0.013882,0.008617,0.006702,0.032073,0.005266,0.006702
3,1.094460,1.051771,0.673025,1.704511,0.238874,0.592189,0.517711,1.271269,0.511656,0.376627,...,0.290039,0.355132,0.717227,0.418105,0.392371,0.585226,0.660914,0.913412,1.101726,1.521344
4,0.251093,0.134916,0.502811,0.120550,1.321674,0.080575,0.194878,0.446596,0.178014,1.098064,...,0.108682,0.073704,0.291068,0.623985,0.090568,0.236727,0.069332,0.147408,0.049344,0.047470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,0.291871,0.210932,1.013315,0.321654,0.297828,0.261388,0.401191,0.625088,0.331114,0.351086,...,0.254730,0.332165,0.925368,0.489488,0.287316,0.257884,0.322705,0.389278,0.343378,0.317449
2860,0.021818,0.025455,0.092468,0.021818,0.015584,0.072727,0.021818,0.121039,0.018701,0.017143,...,0.028052,0.011429,0.339221,0.017143,0.012987,0.020260,0.005714,0.058182,0.018701,0.025455
2861,0.084702,0.059421,0.127269,0.057044,0.015774,0.050562,0.047537,0.124244,0.058773,0.029602,...,0.052723,0.031547,0.173509,0.059637,0.058557,0.064823,0.053803,0.103933,0.076707,0.095938
2862,0.036061,0.021267,0.052011,0.023347,0.010402,0.021960,0.020342,0.081600,0.022654,0.014101,...,0.031438,0.018724,0.522191,0.033287,0.033518,0.032825,0.027739,0.044152,0.018955,0.023810


In [40]:
A = single_cell
b = bulk_values.iloc[:,0]
res = {}
b

0       0.272364
1       0.651938
2       0.014840
3       1.094460
4       0.251093
          ...   
2859    0.291871
2860    0.021818
2861    0.084702
2862    0.036061
2863    0.430997
Name: X1, Length: 2864, dtype: float64

In [43]:
for i in range(89):
    b = bulk_values.iloc[:,i]
    #Use nnls to get initial guess
    x0, rnorm = nnls(A,b)


    #Define minimisation function
    def fn(x, A, b):
        return np.linalg.norm(A.dot(x) - b)

    #Define constraints and bounds
    cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
    bounds = [[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None],[0., None]]

    #Call minimisation subject to these values
    minout = minimize(fn, x0, args=(A, b), method='trust-constr',bounds=bounds,constraints=cons)
    x = minout.x

    res[i] = x,x.sum(),fn(x,A,b)
#print(x,x.sum(),fn(x,A,b))

In [44]:
res

{0: (array([1.13454370e-08, 1.42429876e-01, 5.13846203e-09, 8.57569888e-01,
         5.54116696e-09, 4.68776968e-09, 5.00001716e-09, 4.99447441e-09,
         1.16380035e-07, 4.71609478e-09, 5.02883304e-09, 4.80470915e-09,
         4.62244772e-09, 4.53417182e-09, 4.79086676e-09, 5.00904427e-09,
         1.77959886e-08, 5.12313515e-09, 2.64621915e-08]),
  1.0000000000001483,
  767.6124930275237),
 1: (array([3.12842692e-06, 1.08594614e-06, 1.07299580e-06, 9.99964026e-01,
         1.04139942e-06, 1.25946993e-06, 1.25872835e-06, 1.33880297e-06,
         1.48194844e-05, 1.32697489e-06, 1.03025547e-06, 1.20213144e-06,
         1.38572603e-06, 1.24250689e-06, 1.34865212e-06, 1.03746743e-06,
         1.14905012e-06, 1.05172375e-06, 1.93945310e-07]),
  1.0,
  2625.199584339795),
 2: (array([1.70245224e-05, 4.07362519e-01, 4.43129019e-06, 5.70447184e-01,
         2.18490626e-02, 4.17153994e-06, 4.03026377e-06, 3.91874638e-06,
         1.96913571e-04, 4.17492877e-06, 4.06103847e-06, 4.01763581e-0